In [ ]:
import time
import pandas as pd
import requests
import json
import concurrent.futures
from decouple import config

# Disable warnings
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [ ]:
def get_jsonparsed_data(url):
    """
    Sends a GET request to FMP's Earning Surprise API and returns the resulting data in a dictionary
    """
    # sending get request and saving the response as response object
    response = requests.get(url=url)
    data = json.loads(response.text)
    return data

In [ ]:
# Pull API key from .env file
FMP_API_KEY = config("FMP_API_KEY")
# Read in earnings data
earnings_df = pd.read_csv("filtered_earnings.csv") 
earnings_df.info()

In [ ]:
dates = earnings_df["date"]
print(dates)

In [ ]:
url_list = []
for idx, val in enumerate(dates):
    ticker = earnings_df["symbol"][idx]
    url = "https://financialmodelingprep.com/api/v3/historical-price-full/{}?from={}&to={}&apikey={}".format(ticker, val, val, FMP_API_KEY)
    url_list.append(url)

In [ ]:
# Call FMP API for each URL using Concurrent library
with concurrent.futures.ThreadPoolExecutor() as executor:
    res = [executor.submit(get_jsonparsed_data, url) for url in url_list]
    concurrent.futures.wait(res)

In [ ]:
# Test results
res[3].result()

In [ ]:
df = pd.DataFrame()
for x in range(len(dates)):
    res_df = pd.DataFrame(res[x].result())
    df = pd.concat([df, res_df])

In [ ]:
print(df)

In [ ]:
df.info()